# Методы сегментации клиентов и целевой аудитории:
_____

Данный проект является второй серией из репозитория по продуктовой аналитике и прежде всего хотелось бы рассмотреть для чего нужна сегментация наших пользователей:

- Сегментация — это объединение объектов в группы по общим признакам.

- Сегментация клиентов/ЦА — это объединение клиентской базы или потенциальных потребителей в группы на основе каких-либо признаков (н-р, пола, возраста, количества покупок и тп.)

Зачем бизнесу сегментировать?
____
1. Выбор сегмента до запуска продукта (фокусирование).
2. Маркетинговая оптимизация (ROMI).
3. Выработка стратегии коммуникации.
4. Рост продаж.
5. Персональный подход.
6. Диагностика источников изменений в метриках.
7. Прогнозирование поведения пользователей.

Вопросы, на которые нужно ответить:
____
1. Какие товары и услуги каким из ваших покупателей нужны?
2. Как доносить ценность для разных сегментов?
3. Какие потребители приносят вам больше денег, а на каких вы не зарабатываете?

Как обычно, начнём с выгрузки и предпросмотра наших данных:

In [2]:
import pandas as pd

In [3]:
orders = pd.read_csv("RFM_ht_data.csv",parse_dates=["InvoiceDate"])
orders.head()

/tmp/ipykernel_58912/1418323986.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv("RFM_ht_data.csv",parse_dates=["InvoiceDate"])


,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


#### Описание данных:
____

- InvoiceNo - номер заказа товара;
- CustomerCode - код пользователя;
- InvoiceDate - дата заказа;
- Amount - сумма заказа.

In [4]:
orders.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

Я бы крайне рекомендовал всегда специфицировать тип данных иначе будут проблемы в работе с данными в будущем:

In [5]:
orders["InvoiceNo"] = orders["InvoiceNo"].astype("str")
orders["CustomerCode"] = orders["CustomerCode"].astype("str")

In [6]:
orders.InvoiceDate.describe()

/tmp/ipykernel_58912/2164741630.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  orders.InvoiceDate.describe()


count                  332730
unique                     30
top       2020-09-30 00:00:00
freq                    19781
first     2020-09-01 00:00:00
last      2020-09-30 00:00:00
Name: InvoiceDate, dtype: object

In [7]:
orders.shape

(332730, 4)

Как видно датасет по времени недолгий, но значений в нём довольно прилично, следовательно это может быть какая-то средняя компания по продажам услуг или товаров.

Также хотелось бы проверить сколько уникальных значений пользователей у нас будет перед и после преобразований фрейма, вам тоже рекомендую так делать.

In [8]:
orders.CustomerCode.nunique()

123733

#### Какие бывают виды сегментации ?
____

1. Методика 5W Марка Шерингтона:
    1. What (Что?) — Какой продукт вы предлагаете целевой аудитории?
    2. Who (Кто?) — Кто покупатель продукта?
    3. Why (Почему?) — Какую потребность удовлетворяет продукт?
    4. When (Когда?) — В какие моменты совершается покупка?
    5. Where (Где?) — В каких точках контакта принимается и реализуется решение о покупке?

2. Khramatrix:
    - Khramatrix — расширенная модель 5W. 4 дополнительных пункта:
Описание целевой аудитории (ЦА) по географическим и демографическим критериям.
Поведенческие особенности ЦА.
Готовность к покупке: готов приобрести, собирает информацию, сравнивает варианты, хочет попробовать.
Целевое действие, к которому Вы должны подтолкнуть ЦА.

- При этом данные сегментации чаще всего используют маркетологи, так что теперь давайте изучим самые популярные виды среди аналитиков:
____
1. RFM-анализ:
Способ сегментации по 3-м показателям:
    1. Recency (давность)
    2. Frequency (частота)
    3. Monetary (деньги)

<p align="center">
  <img  src="rfm.png">
</p>

2. Когортный анализ
    - Метод анализа поведения пользователей на основе объединения их по какому-либо признаку во времени.

<p align="center">
  <img  src="cohort_analysis.png">
</p>


При этом последний метод я очень подробно описал в этом [докладе](https://github.com/0n1xx/RFM_Cohort_Analysis), в котором совсем чуть-чуть затронул и RFM-анализ, так вот давайте погрузимся в него поглубже и ответим на несколько вопросов после сегментирования.
Для тех кто совсем не знаком с RFM-анализом рекомендую почитать вот эту [статью](https://aainabajaj39.medium.com/rfm-analysis-for-successful-customer-segmentation-using-python-6291decceb4b).

In [10]:
last_date = orders["InvoiceDate"].max()
last_date

Timestamp('2020-09-30 00:00:00')

In [11]:
rfmTable = orders.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days,
                                               # Recency #Количество дней с последнего заказа
                                           'InvoiceNo': lambda x: len(x),
                                               # Frequency #Количество заказов
                                           'Amount': lambda x: x.sum()})
                                               # Monetary Value #Общая сумма по всем заказам

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency',
                         'InvoiceNo': 'frequency',
                         'Amount': 'monetary_value'}, inplace=True)

In [12]:
rfmSegmentation = rfmTable

In [13]:
quantiles = rfmSegmentation.quantile(q=[0.25,0.5,0.75])

Разделение по RFM мы можем делать на сколько угодно квантилей, а порой стоит делить и по значениям, но в этом докладе я использую деление по 25% процентилю:

In [15]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]:
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]:
        return 2
    else:
        return 1


In [16]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str)                             + rfmSegmentation.F_Quartile.map(str)                             + rfmSegmentation.M_Quartile.map(str)

In [17]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


In [18]:
rfmSegmentation.shape[0]

123733

Проверка на соотношение пользователей прошла успешна и мы не потеряли никого, теперь можно начать отвечать на вопросы:

### Часть с вопросами:
____

1. Какое максимальное кол-во покупок было совершено одним пользователем ?
____

In [20]:
rfmSegmentation.frequency.max()

204

2. Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М) ?
____

In [21]:
quantiles.monetary_value[0.25]

765.0

3. Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F ?
____

In [22]:
rfmSegmentation.query("F_Quartile== 1").frequency.min()

4

4. Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R ?
____

In [23]:
rfmSegmentation.query("R_Quartile == 2").recency.max()

8

5. Сколько пользователей попало в сегмент 111 ?
____

In [26]:
rfmSegmentation.RFMClass = rfmSegmentation["RFMClass"].astype("int")

In [27]:
rfmSegmentation.query("RFMClass == 111").shape[0]

9705

6. Сколько пользователей попало в сегмент 311 ?
____

In [115]:
rfmSegmentation.query("RFMClass == 311").shape[0]

1609

7. В каком RFM-сегменте самое большое кол-во пользователей ?
8. В каком RFM-сегменте самое маленькое кол-во пользователей ?
9. Какое количество пользователей попало в самый малочисленный сегмент ?
____

In [28]:
rfmSegmentation.reset_index(inplace=True)

In [29]:
rfmSegmentation.groupby("RFMClass").agg({"CustomerCode":"count"}).sort_values("CustomerCode",ascending=False)

,CustomerCode
RFMClass,
444,10624
111,9705
443,6729
344,6593
211,5847
...,...
424,63
214,60
114,60


Ответы:
____
7. В сегменте 444 находиться больше всего пользователей;
8. В сегменте 414 находиться меньше всего пользователей;
9. В самый малочисленный сегмент попало всего лишь 2 пользователя.

#### Выводы:
____

- В данном докладе мы узнали про методы сегментации, а также применили RFM-анализ на практике.